In [1]:
import random 
import pickle

import numpy as np 
import pandas as pd 
from nltk.tokenize import RegexpTokenizer

from tensorflow.keras.models import Sequential, load_model 
from tensorflow.keras.layers import LSTM, Dense, Activation 
from tensorflow.keras.optimizers import RMSprop

In [2]:
text_df = pd.read_csv("zinnen1.csv")

In [3]:
text = list(text_df.text.values)
joined_text = " ".join(text)

In [4]:
partial_text = joined_text[:20000]

In [5]:
tokenizer = RegexpTokenizer(r"\w+")
tokens = tokenizer.tokenize(partial_text.lower())

In [6]:
unique_tokens = np.unique(tokens)
unique_token_index = {token: idx for idx, token in enumerate(unique_tokens)}

In [7]:
n_words = 10
input_words = []
next_words = []

for i in range(len(tokens) - n_words):
    input_words.append(tokens[i:i + n_words])
    next_words.append(tokens[i + n_words])

In [8]:
X = np.zeros((len(input_words), n_words, len(unique_tokens)), dtype=bool)
y = np.zeros((len(next_words), len(unique_tokens)), dtype=bool)

In [9]:
for i, words in enumerate(input_words):
    for j, word in enumerate(words):
        X[i, j, unique_token_index[word]] = 1
    y[i, unique_token_index[next_words[i]]] = 1

In [10]:
model = Sequential()
model.add(LSTM(128, input_shape=(n_words, len(unique_tokens)), return_sequences=True))
model.add(LSTM(128))
model.add(Dense(len(unique_tokens)))
model.add(Activation("softmax"))

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer=RMSprop(learning_rate=0.01), metrics=["accuracy"])
model.fit(X, y, batch_size=128, epochs=30, shuffle=True)

Epoch 1/30


27/27 [==============================] - 4s 31ms/step - loss: 6.0010 - accuracy: 0.0750
Epoch 2/30
27/27 [==============================] - 1s 30ms/step - loss: 5.7032 - accuracy: 0.0834
Epoch 3/30
27/27 [==============================] - 1s 31ms/step - loss: 5.5560 - accuracy: 0.0822
Epoch 4/30
27/27 [==============================] - 1s 39ms/step - loss: 5.2716 - accuracy: 0.0890
Epoch 5/30
27/27 [==============================] - 1s 48ms/step - loss: 4.9577 - accuracy: 0.1155
Epoch 6/30
27/27 [==============================] - 1s 52ms/step - loss: 4.7107 - accuracy: 0.1370
Epoch 7/30
27/27 [==============================] - 1s 47ms/step - loss: 4.3196 - accuracy: 0.1689
Epoch 8/30
27/27 [==============================] - 1s 53ms/step - loss: 3.9762 - accuracy: 0.1992
Epoch 9/30
27/27 [==============================] - 1s 52ms/step - loss: 3.6675 - accuracy: 0.2448
Epoch 10/30
27/27 [==============================] - 1s 52ms/step - loss: 3.3501 - accuracy: 0.2993
Epoch 1

In [ ]:
model.save("my_model.keras")

In [ ]:
model = load_model("my_model.keras")

In [ ]:
def predict_next_word(input_text, n_best):
    input_text = input_text.lower()
    X = np.zeros((1, n_words, len(unique_tokens)))
    for i, word in enumerate(input_text.split()):
        X[0, i, unique_token_index[word]] = 1
        
    predictions = model.predict(X)[0]
    return np.argpartition(predictions, -n_best)[-n_best:]

In [ ]:
possible = predict_next_word("De maan lijkt", 5)

In [ ]:
print([unique_tokens[idx] for idx in possible])

In [ ]:
def generate_text(input_text, text_lenght, creativity=3):
    word_sequence = input_text.split()
    current = 0
    for _ in range(text_lenght):
        sub_sequence = " ".join(tokenizer.tokenize(" ".join(word_sequence).lower())[current:current+n_words])
        try:
            choice = unique_tokens[random.choice(predict_next_word(sub_sequence, creativity))]
        except:
            choice = random.choice(unique_tokens)
        word_sequence.append(choice)
        current += 1

    
    return " ".join(word_sequence)

In [ ]:
generate_text("De maan lijkt", 10, 5)